# D.C. Residential Properties
Residential Properties in Washington D.C.

**About this file**
Dataset ready for analysis showing real property information, including most recent sales price as of July 2018, for properties located Washington, D.C.


**Columns in this Dataset**

|Column Name|Description|Type|
|--|--|--|
|GBA|Gross building area in square feet|Int|
|BLDG_NUM|Building Number on Property|Int|
|STYLE|Style|String|
|STRUCT|Structure|String|
|GRADE|Grade|String|
|CNDTN|Condition|String|
|EXTWALL|Extrerior wall|String|
|ROOF|Roof type|String|
|INTWALL|Interior wall|String|
|KITCHENS|Number of kitchens|Int|
|FIREPLACES|Number of fireplaces|Int|
|LANDAREA|Last Modified Date|Int|
|GIS_LAST_MOD_DTTM|Condition|Timestamp|
|SOURCE|Raw Data Source|String|
|CMPLX_NUM|Complex number|String|


In [1]:
data = "gs://is843_bigdaddydata/notebooks/data/"